In [ ]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *
from plotnine.options import set_option

In [ ]:
def auc_barplot(df: pl.DataFrame, x_col: str, y_col: str, model_col: str, facet_col: str, title: str = None, subtitle: str = None, x_axis_label: str = None, y_axis_label: str = None, text_scale: float = 1, legend_rows: int = 1) -> ggplot:
    
    p = (
		ggplot(df, aes(x=x_col, y=y_col, fill=model_col))
		+ geom_col(position=position_dodge(), color="black")
		+ facet_wrap(facet_col, ncol=1, scales="free_x")
		+ labs(
			title = title,
			subtitle = subtitle,
			x = x_axis_label,
			y = y_axis_label
		)
		+ theme_minimal()
		+ theme(
			axis_text_x=element_text(size=8*text_scale, angle=30, hjust = 1, vjust = 1),
			axis_ticks=element_line(),
			plot_title=element_text(size=20*text_scale, face="bold", margin = {'t': 5*text_scale}, hjust=0.5),
			plot_subtitle=element_text(margin = {'b': 10*text_scale}, hjust=0.5),
			axis_title_x=element_text(size=16*text_scale),
			axis_title_y=element_text(size=16*text_scale),
			axis_text_y=element_text(size=12*text_scale),
			legend_title=element_blank(),
			legend_text=element_text(size=12*text_scale),
			strip_text=element_text(size=14*text_scale),
			panel_grid_major=element_blank(),
			panel_grid_minor=element_blank(),
			# Positioning
			legend_position = "bottom",
			legend_key_size = 15*text_scale,
		)
		+ guides(fill=guide_legend(nrow=legend_rows))
		+ scale_y_continuous(limits=(0, 1))
	)
    
    return p

In [ ]:
stylized_model_names = {
	"microsec" : "MicroSEC",
	"mobsnvf" : "MOBSNVF",
	"vafsnvf" : "VAFSNVF",
	"sobdetector" : "SOBDetector",
	"gatk-obmm": "GATK-OBMM"
}

stylized_vcaller_names = {
	"muse" : "MuSE",
	"mutect2" : "MuTect2",
	"somaticsniper" : "SomaticSniper",
	"varscan2" : "VarScan2"
}


### TCGA

In [ ]:
tcga_annot = pl.read_csv("../../analysis-tcga-ffpe/gdc/annot2/vcf_annotation.tsv", separator="\t")

#### Filtered FFPE VCF vs Filtered Fresh-Frozen VCF | C:G>T:A SNVs only

In [ ]:
paths = [
	"tcga/tcga.muse.matched.f-ffpe_f-ff.ct-snvs.tsv",
	"tcga/tcga.mutect2.matched.f-ffpe_f-ff.ct-snvs.tsv",
	"tcga/tcga.somaticsniper.matched.f-ffpe_f-ff.ct-snvs.tsv",
	"tcga/tcga.varscan2.matched.f-ffpe_f-ff.ct-snvs.tsv"
]

tcga_matched_f_f_ct = []

for path in paths:

	tcga_matched_f_f_ct_caller = pl.read_csv(path, separator="\t").with_columns(wtype=pl.lit(stylized_vcaller_names.get(path.split(".")[1], path.split(".")[1])))
	tcga_matched_f_f_ct.append(tcga_matched_f_f_ct_caller)

tcga_matched_f_f_ct = pl.concat(tcga_matched_f_f_ct)
tcga_matched_f_f_ct = tcga_matched_f_f_ct.with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
tcga_matched_f_f_ct = tcga_matched_f_f_ct.filter(~pl.col("sample_name").str.contains("all-"))

tcga_matched_f_f_ct.write_csv("tcga/tcga.matched.f-ffpe_f-ff.ct-snvs.tsv", separator="\t")

display(tcga_matched_f_f_ct)

set_option("figure_size", (16, 16))

tcga_matched_f_f_ct_auroc_barplot = auc_barplot(
	tcga_matched_f_f_ct,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "TCGA",
	subtitle = "Filtered FFPE VCF & Filtered Fresh-Frozen VCF",
	x_axis_label="Sample",
	y_axis_label="AUROC"
)

tcga_matched_f_f_ct_auroc_barplot.save("tcga.auroc.barplot.matched.f-ffpe_f-ff.ct-snvs.pdf")
display(tcga_matched_f_f_ct_auroc_barplot)


tcga_matched_f_f_ct_auprc_barplot = auc_barplot(
	tcga_matched_f_f_ct,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "TCGA",
	subtitle = "Filtered FFPE VCF & Filtered Fresh-Frozen VCF",
	x_axis_label="Sample",
	y_axis_label="AUPRC"
)

tcga_matched_f_f_ct_auprc_barplot.save("tcga.auprc.barplot.matched.f-ffpe_f-ff.ct-snvs.pdf")
tcga_matched_f_f_ct_auprc_barplot

#### Filtered FFPE VCF vs Filtered Fresh-Frozen VCF | All SNVs

In [ ]:
tcga_matched_f_f_all = pl.read_csv("tcga/tcga.matched.f-ffpe_f-ff.all-snvs.tsv", separator="\t").filter(~pl.col("sample_name").str.contains("all-"))
tcga_matched_f_f_all

In [ ]:
set_option("figure_size", (16, 16))

tcga_matched_f_f_all_barplot = auc_barplot(
	tcga_matched_f_f_all,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="workflow_type",
	title = "TCGA",
	subtitle = "Filtered FFPE VCF & Filtered Fresh-Frozen VCF | All SNVs Evaluated for SOBDetector",
	x_axis_label="Sample",
	y_axis_label="AUROC"
)
tcga_matched_f_f_all_barplot.save("tcga.auroc.barplot.matched.f-ffpe_f-ff.all-snvs.pdf")
tcga_matched_f_f_all_barplot

In [ ]:
set_option("figure_size", (16, 16))

tcga_matched_f_f_all_barplot = auc_barplot(
	tcga_matched_f_f_all,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="workflow_type",
	title = "TCGA",
	subtitle = "Filtered FFPE VCF & Filtered Fresh-Frozen VCF | All SNVs Evaluated for SOBDetector",
	x_axis_label="Sample",
	y_axis_label="AUPRC"
)
tcga_matched_f_f_all_barplot.save("tcga.auprc.barplot.matched.f-ffpe_f-ff.all-snvs.pdf")
tcga_matched_f_f_all_barplot

#### Unfiltered FFPE VCF vs Unfiltered Fresh-Frozen VCF | All SNVs

In [ ]:
tcga_matched_uf_uf_ct = (
    pl.read_csv("tcga/tcga.matched.uf-ffpe_uf-ff.ct-snvs.tsv", separator="\t")
    .filter(~pl.col("vcf_fid").str.contains("all-"))
    .with_columns(pl.col("wtype").map_elements(lambda x : stylized_vcaller_names.get(x, x), return_dtype=str))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)
tcga_matched_uf_uf_ct

In [ ]:
set_option("figure_size", (16, 16))

tcga_matched_uf_uf_ct_barplot = auc_barplot(
	tcga_matched_uf_uf_ct,
	x_col="vcf_fid",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "TCGA",
	subtitle = "Unfiltered FFPE VCF & Unfiltered Fresh-Frozen VCF | C>T SNVs Evaluated",
	x_axis_label="Sample",
	y_axis_label="AUROC"
)
tcga_matched_uf_uf_ct_barplot.save("tcga.auroc.barplot.matched.uf-ffpe_uf-ff.ct-snvs.pdf")
tcga_matched_uf_uf_ct_barplot

In [ ]:
set_option("figure_size", (16, 16))

tcga_matched_uf_uf_ct_barplot = auc_barplot(
	tcga_matched_uf_uf_ct,
	x_col="vcf_fid",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "TCGA",
	subtitle = "Unfiltered FFPE VCF & Unfiltered Fresh-Frozen VCF | C>T SNVs Evaluated",
	x_axis_label="Sample",
	y_axis_label="AUPRC"
)
tcga_matched_uf_uf_ct_barplot.save("tcga.auprc.barplot.matched.uf-ffpe_uf-ff.ct-snvs.pdf")
tcga_matched_uf_uf_ct_barplot

### ENA

#### ENA PRJEB8754

In [ ]:
ena_PRJEB8754_f_f_ct_aucs =  (
    pl.read_csv("ena_PRJEB8754/ena.PRJEB8754.tumor-only.f-ffpe_f-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 Tumor-only"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(sample_name=pl.col("sample_name").map_elements(lambda x : "_".join(x.split("_")[:3]), return_dtype=str))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

set_option("figure_size", (5, 4))

ena_PRJEB8754_f_f_ct_auroc_barplot = auc_barplot(
	ena_PRJEB8754_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "Betge15",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)
ena_PRJEB8754_f_f_ct_auroc_barplot.save("ena.PRJEB8754.auroc.barplot.tumor-only.f-ffpe_f-ff.pdf")
ena_PRJEB8754_f_f_ct_auroc_barplot

In [ ]:
ena_PRJEB8754_f_f_ct_auprc_barplot = auc_barplot(
	ena_PRJEB8754_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "Betge15",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

ena_PRJEB8754_f_f_ct_auprc_barplot.save("ena.PRJEB8754.auprc.barplot.tumor-only.f-ffpe_f-ff.pdf")
ena_PRJEB8754_f_f_ct_auprc_barplot

In [ ]:
ena_PRJEB44073_f_f_ct_aucs =  (
    pl.read_csv("ena_PRJEB44073/ena.PRJEB44073.tumor-only.f-ffpe_f-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 Tumor-only"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(ena_PRJEB44073_f_f_ct_aucs)

set_option("figure_size", (5, 4))

ena_PRJEB44073_f_f_ct_auroc_barplot = auc_barplot(
	ena_PRJEB44073_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "Chong21",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)
ena_PRJEB44073_f_f_ct_auroc_barplot.save("ena.PRJEB44073.auroc.barplot.tumor-only.f-ffpe_f-ff.pdf")
ena_PRJEB44073_f_f_ct_auroc_barplot

In [ ]:
ena_PRJEB44073_f_f_ct_auprc_barplot = auc_barplot(
	ena_PRJEB44073_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "Chong21",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)
ena_PRJEB44073_f_f_ct_auprc_barplot.save("ena.PRJEB44073.auprc.barplot.tumor-only.f-ffpe_f-ff.pdf")
ena_PRJEB44073_f_f_ct_auprc_barplot

In [ ]:
ena_SRP044740_f_f_ct_aucs =  (
    pl.read_csv("ena_SRP044740/ena.SRP044740.tumor-only.f-ffpe_f-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 Tumor-only"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(ena_SRP044740_f_f_ct_aucs)

set_option("figure_size", (9, 5))

ena_SRP044740_f_f_ct_auroc_barplot = auc_barplot(
	ena_SRP044740_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "ENA SRP044740",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)

ena_SRP044740_f_f_ct_auroc_barplot.save("ena.SRP044740.auroc.barplot.tumor-only.f-ffpe_f-ff.pdf")
ena_SRP044740_f_f_ct_auroc_barplot

In [ ]:
ena_SRP044740_f_f_ct_auprc_barplot = auc_barplot(
	ena_SRP044740_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "ENA SRP044740",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

ena_SRP044740_f_f_ct_auprc_barplot.save("ena.SRP044740.auprc.barplot.tumor-only.f-ffpe_f-ff.pdf")
ena_SRP044740_f_f_ct_auprc_barplot

In [ ]:
ena_SRP065941_f_f_ct_aucs =  (
    pl.read_csv("ena_SRP065941/ena.SRP065941.matched.f-ffpe_f-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 matched-normal"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(ena_SRP065941_f_f_ct_aucs)

set_option("figure_size", (5, 4))

ena_SRP065941_f_f_ct_auroc_barplot = auc_barplot(
	ena_SRP065941_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "oh15",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)

ena_SRP065941_f_f_ct_auroc_barplot.save("ena.SRP065941.auroc.barplot.matched.f-ffpe_f-ff.pdf")
ena_SRP065941_f_f_ct_auroc_barplot

In [ ]:
ena_SRP065941_f_f_ct_auprc_barplot = auc_barplot(
	ena_SRP065941_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "oh15",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

ena_SRP065941_f_f_ct_auprc_barplot.save("ena.SRP065941.auprc.barplot.matched.f-ffpe_f-ff.pdf")
ena_SRP065941_f_f_ct_auprc_barplot

### EGA

In [ ]:
ega_EGAD00001004066_f_f_ct_aucs =  (
    pl.read_csv("ega_EGAD00001004066/ega.EGAD00001004066.matched.f-ffpe_f-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 matched-normal"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(ega_EGAD00001004066_f_f_ct_aucs)

set_option("figure_size", (8, 5))

ega_EGAD00001004066_f_f_ct_auroc_barplot = auc_barplot(
	ega_EGAD00001004066_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "bonnet18",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)

ega_EGAD00001004066_f_f_ct_auroc_barplot.save("ega.EGAD00001004066.auroc.barplot.matched.f-ffpe_f-ff.pdf")
ega_EGAD00001004066_f_f_ct_auroc_barplot

In [ ]:
ega_EGAD00001004066_f_f_ct_auprc_barplot = auc_barplot(
	ega_EGAD00001004066_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "bonnet18",
	subtitle = " ",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

ega_EGAD00001004066_f_f_ct_auprc_barplot.save("ega.EGAD00001004066.auprc.barplot.matched.f-ffpe_f-ff.pdf")
ega_EGAD00001004066_f_f_ct_auprc_barplot

### SEQC2

In [ ]:
seqc2_ffg_f_f_ct_aucs =  (
    pl.read_csv("seqc2/seqc2.ffg.matched.f-ffpe_f-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 matched-normal"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(seqc2_ffg_f_f_ct_aucs)

set_option("figure_size", (5, 4))

seqc2_ffg_f_uf_ct_auroc_barplot = auc_barplot(
	seqc2_ffg_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFG",
	subtitle = "Filtered FFPE VCF and Filtered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffg_f_uf_ct_auroc_barplot.save("seqc2.ffg.auroc.barplot.matched.f-ffpe_f-ff.pdf")
seqc2_ffg_f_uf_ct_auroc_barplot

In [ ]:
seqc2_ffg_f_uf_ct_auroc_barplot = auc_barplot(
	seqc2_ffg_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFG",
	subtitle = "Filtered FFPE VCF and Filtered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffg_f_uf_ct_auroc_barplot.save("seqc2.ffg.auprc.barplot.matched.f-ffpe_f-ff.pdf")
seqc2_ffg_f_uf_ct_auroc_barplot

In [ ]:
seqc2_ffg_f_uf_ct_aucs =  (
    pl.read_csv("seqc2/seqc2.ffg.matched.f-ffpe_uf-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 matched-normal"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(seqc2_ffg_f_uf_ct_aucs)

set_option("figure_size", (5, 4))

seqc2_ffg_f_uf_ct_auroc_barplot = auc_barplot(
	seqc2_ffg_f_uf_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFG",
	subtitle = "Filtered FFPE VCF and Unfiltered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffg_f_uf_ct_auroc_barplot.save("seqc2.ffg.auroc.barplot.matched.f-ffpe_uf-ff.pdf")
seqc2_ffg_f_uf_ct_auroc_barplot

In [ ]:
seqc2_ffg_f_uf_ct_auprc_barplot = auc_barplot(
	seqc2_ffg_f_uf_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFG",
	subtitle = "Filtered FFPE VCF and Unfiltered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffg_f_uf_ct_auprc_barplot.save("seqc2.ffg.auprc.barplot.matched.f-ffpe_uf-ff.pdf")
seqc2_ffg_f_uf_ct_auprc_barplot

In [ ]:
seqc2_ffx_f_f_ct_aucs =  (
    pl.read_csv("seqc2/seqc2.ffx.matched.f-ffpe_f-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 matched-normal"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(seqc2_ffx_f_f_ct_aucs)

set_option("figure_size", (5, 4))

seqc2_ffx_f_f_ct_auroc_barplot = auc_barplot(
	seqc2_ffx_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFX",
	subtitle = "Filtered FFPE VCF and Filtered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffx_f_f_ct_auroc_barplot.save("seqc2.ffx.auroc.barplot.matched.f-ffpe_f-ff.pdf")
seqc2_ffx_f_f_ct_auroc_barplot

In [ ]:
seqc2_ffx_f_f_ct_auprc_barplot = auc_barplot(
	seqc2_ffx_f_f_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFX",
	subtitle = "Filtered FFPE VCF and Filtered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffx_f_f_ct_auprc_barplot.save("seqc2.ffx.auprc.barplot.matched.f-ffpe_f-ff.pdf")
seqc2_ffx_f_f_ct_auprc_barplot

In [ ]:
seqc2_ffx_f_uf_ct_aucs =  (
    pl.read_csv("seqc2/seqc2.ffx.matched.f-ffpe_uf-ff.tsv", separator="\t")
    .with_columns(wtype = pl.lit("MuTect2 matched-normal"))
    # .filter(~pl.col("sample_name").str.contains("all-"))
    .with_columns(pl.col("model").map_elements(lambda x : stylized_model_names.get(x, x), return_dtype=str))
)

display(seqc2_ffx_f_uf_ct_aucs)

set_option("figure_size", (5, 4))

seqc2_ffx_f_uf_ct_auroc_barplot = auc_barplot(
	seqc2_ffx_f_uf_ct_aucs,
	x_col="sample_name",
	y_col="auroc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFX",
	subtitle = "Filtered FFPE VCF and Unfiltered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUROC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffx_f_uf_ct_auroc_barplot.save("seqc2.ffx.auroc.barplot.matched.f-ffpe_uf-ff.pdf")
seqc2_ffx_f_uf_ct_auroc_barplot

In [ ]:
seqc2_ffx_f_uf_ct_auprc_barplot = auc_barplot(
	seqc2_ffx_f_uf_ct_aucs,
	x_col="sample_name",
	y_col="auprc",
	model_col="model",
	facet_col="wtype",
	title = "SEQC2 FFX",
	subtitle = "Filtered FFPE VCF and Unfiltered FF VCF",
	x_axis_label="Sample",
	y_axis_label="AUPRC",
	text_scale=1,
	legend_rows=2
)

seqc2_ffx_f_uf_ct_auprc_barplot.save("seqc2.ffx.auprc.barplot.matched.f-ffpe_uf-ff.pdf")
seqc2_ffx_f_uf_ct_auprc_barplot